In [1]:
#%pip install keras-tuner

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to between 0 and 1

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Function to create a model for hyperparameter tuning
def build_model(hp):
    model = Sequential()
    

     # Add the first hidden layer with neurons varying between 250 to 260 with a step value of 2
    hp_neurons = hp.Int('neurons', min_value=250, max_value=260, step=2)
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(units=hp_neurons, activation='sigmoid'))

    # Add the second hidden layer with 16 neurons and activation function tanh
    model.add(Dense(16, activation='tanh'))

    # Add the third hidden layer with 8 neurons and activation function relu
    model.add(Dense(8, activation='relu'))

    # Add the fourth hidden layer with 4 neurons and activation function selu
    model.add(Dense(4, activation='selu'))

    model.add(Dense(10, activation='softmax'))

    # Hyperparameters to tune
    learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.15])
    # Compile model
    model.compile(optimizer=SGD(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Instantiate the KerasTuner RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='tuner_logs',
    project_name='mnist_tuning'
)

hp_batch_size = hp.Choice('batch_size', values=[4, 8, 16])
# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=hp_batch_size)

# Get the best hyperparameters
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters:")
print(best_hps)

# Build the model with the best hyperparameters
best_model = tuner.oracle.get_best_trials(num_trials=1)[0].hypermodel.build(best_hps)

# Train the best model on the entire training set
best_model.fit(x_train, y_train, epochs=10, batch_size=best_hps['batch_size'], validation_split=0.1)

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {test_acc}")


NameError: name 'hp' is not defined

In [ ]:
#  Get the best hyperparameters
# best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
# print("Best Hyperparameters:")
# print(best_hps)

# # Build the model with the best hyperparameters
# best_model = tuner.oracle.get_best_trials(num_trials=1)[0].hypermodel.build(best_hps)

# # Train the best model on the entire training set
# best_model.fit(x_train, y_train, epochs=10, batch_size=best_hps['batch_size'], validation_split=0.1)

# # Evaluate the best model on the test set
# test_loss, test_acc = best_model.evaluate(x_test, y_test)
# print(f"\nTest Accuracy: {test_acc}")